# RFs and DL2 (data and MCs) scripts

In this notebook, you can try running magic-cta-pipe scripts on a tiny data sample. Because of time constraints, you could not run the pipeline on all the files needed to produce plots (e.g. SED), so we will provide you a complete dataset to get to 'nice' plots and a few MCs and data *.h5* files to run the pipeline.
You have to provide the MCP scripts path ana paths (+filenames) where you want to save logging infos.

In [45]:
import glob
import logging
import os
import subprocess
import sys
log = logging.getLogger()

# RFs training

Here you will train RFs: two regressors (energy and direction reconstruction) and one classifier (gamma/hadron separation).

Events are separated according to their combo-type (LST1+M1, LST1+M2, M1+M2, LST1+M1+M2) and these subsamples are used to train telescope-wise RFs. So you will get e.g. 4 classifier output files (4 combo-types) and in each file you will find one RF every telescope 

Events used to train RFs are extracted in a random way from the files in the input folders. Gammas are used to train all the RFs types, protons are only used to train classifiers

The script runs some of the Python Scikit-learn functions.

Options:

-g: MC stereo dl1 gammas directory (diffuse)

-p: MC stereo dl1 protons directory (diffuse)

-o: directory to save the output

-c: configuration file:
1) gamma_offaxis: diffuse gammas direction ring selection (null=no cuts on their direction; here we use a (slightly asymmetric to compensate for different statistics) ring around 0.4 deg (see Crab camera position in wobble mode) 

train-\*: to train the energy regressor, the disp regressor or the classifier; you can train one, two or every RF when you launch the script, but you will need all the RFs (energy, disp, classifier) for the next step (Dl1 to Dl2)

use-unsigned: use features absolute value to train RFs 

Warning: In case of real training, you will need a lot of proton and gammas runs for every pointing direction, so it would be better to merge the input runs (one *.h5* every pointing)


In [46]:
%%capture
os.system("sed -n '80,200p' /fefs/aswg/workspace/elisa.visentin/sw_school/config_dyn.yaml")



energy_regressor:
    settings:
        n_estimators: 150
        criterion: "squared_error"
        max_depth: 50
        min_samples_split: 2
        min_samples_leaf: 2
        min_weight_fraction_leaf: 0.0
        max_features: 1.0
        max_leaf_nodes: null
        min_impurity_decrease: 0.0
        bootstrap: true
        oob_score: false
        n_jobs: 5
        random_state: 42
        verbose: 0
        warm_start: false
        ccp_alpha: 0.0
        max_samples: null

    features: [
        "intensity",
        "length",
        "width",
        "skewness",
        "kurtosis",
        "slope",
        "intensity_width_2",
        "h_max",
        "impact",
        "pointing_alt",
        "pointing_az",
    ]

    gamma_offaxis:
        min: 0.2 deg
        max: 0.5 deg


disp_regressor:
    settings:
        n_estimators: 150
        criterion: "squared_error"
        max_depth: 50
        min_samples_split: 2
        min_samples_leaf: 2
        min_weight_fraction_lea

**Settings**

In [47]:
f=open('/fefs/aswg/workspace/elisa.visentin/sw_school/RF.log','w')

scripts=('/home/elisa.visentin/magic-cta-pipe/magicctapipe/scripts/lst1_magic/')
input_dir_gamma=('/fefs/aswg/workspace/elisa.visentin/sw_school/DL1_stereo/gamma/train/')
input_dir_proton=('/fefs/aswg/workspace/elisa.visentin/sw_school/DL1_stereo/proton/train/')
output_dir_rf=('/fefs/aswg/workspace/elisa.visentin/sw_school')
config=('/fefs/aswg/workspace/elisa.visentin/sw_school/config_dyn.yaml')

cd to the scripts directory to launch them

In [48]:
cd $scripts

/fefs/home/elisa.visentin/magic-cta-pipe/magicctapipe/scripts/lst1_magic


Here we use python subprocess.run to run the script and get a log file

In [49]:
a=subprocess.run(['python','lst1_magic_train_rfs.py', f'-g{input_dir_gamma}', f'-p{input_dir_proton}',\
    f'-o{output_dir_rf}', f'-c{config}', '--train-energy', '--train-disp', '--train-classifier',\
        '--use-unsigned'], stdout=f, stderr=f) 

Here you can check your log file

In [50]:
more /fefs/aswg/workspace/elisa.visentin/sw_school/RF.log

# DL1 to DL2 

Here you will use your RFs to get to DL2 MCs and real data: DL2 data contain events whose energy, direction and gammaness are evaluated by RFs telescope-wise (so if three telescopes see the same event, this event will have three energy/direction/gammaness values (LSt1/M1/M2)).

As for RFS, the events are separated in combo_types and the right combo RFs are applied.

To reconstruct the arrival directions, MCP script uses the MARS-like DISP
method(i.e. look for the minimum angular distance between all the head and tail candidates)

Options:

-d: input file (dl1 stereo, MCs or real data; test sample gammas are ring-wobble (0.4 deg), test sample protons are diffuse)

-r: directory where you stored your RFs

-o: output directory, to store DL2 files



### MCs

**Settings**

In [51]:
f=open('/fefs/aswg/workspace/elisa.visentin/sw_school/DL2_mc.log','w')

scripts=('/home/elisa.visentin/magic-cta-pipe/magicctapipe/scripts/lst1_magic/')
input_gamma=('/fefs/aswg/workspace/elisa.visentin/sw_school/DL1_stereo/gamma/test/*.h5')
input_proton=('/fefs/aswg/workspace/elisa.visentin/sw_school/DL1_stereo/proton/test/*.h5')
input_dir_rf=('/fefs/aswg/workspace/elisa.visentin/sw_school/')
output_dir_dl2=('/fefs/aswg/workspace/elisa.visentin/sw_school/')

cd to the scripts directory to launch them

In [52]:
cd $scripts

/fefs/home/elisa.visentin/magic-cta-pipe/magicctapipe/scripts/lst1_magic


Lines to get files from gamma/proton folders 

In [53]:
input_file_gamma = glob.glob(input_gamma)
input_file_gamma.sort()
input_file_proton = glob.glob(input_proton)
input_file_proton.sort()

Here we use python subprocess.run to run the script and get a log file

In [54]:
for input_file in input_file_gamma: 
    b=subprocess.run(['python','lst1_magic_dl1_stereo_to_dl2.py', f'-d{input_file}', f'-r{input_dir_rf}',\
        f'-o{output_dir_dl2}'], stdout=f, stderr=f)     
        
for input_file in input_file_proton:
    c=subprocess.run(['python','lst1_magic_dl1_stereo_to_dl2.py', f'-d{input_file}', f'-r{input_dir_rf}',\
        f'-o{output_dir_dl2}'], stdout=f, stderr=f)

Here you can check your log file

In [55]:
more /fefs/aswg/workspace/elisa.visentin/sw_school/DL2_mc.log

### Real data

**Settings**

In [56]:
f=open('/fefs/aswg/workspace/elisa.visentin/sw_school/DL2_data.log','w')

input_data=('/fefs/aswg/workspace/elisa.visentin/sw_school/DL1_stereo/real/*.h5')
input_dir_rf=('/fefs/aswg/workspace/elisa.visentin/sw_school/')
output_dir_dl2=('/fefs/aswg/workspace/elisa.visentin/sw_school/')

cd to the scripts directory to launch them

In [57]:
cd $scripts

/fefs/home/elisa.visentin/magic-cta-pipe/magicctapipe/scripts/lst1_magic


Lines to get files from gamma/proton folders 

In [58]:
input_file_data = glob.glob(input_data)
input_file_data.sort()

Here we use python subprocess.run to run the script and get a log file

In [59]:
for input_file in input_file_data: 
    d=subprocess.run(['python','lst1_magic_dl1_stereo_to_dl2.py', f'-d{input_file}', f'-r{input_dir_rf}',\
        f'-o{output_dir_dl2}'], stdout=f, stderr=f)     

Here you can check your log file

In [60]:
more /fefs/aswg/workspace/elisa.visentin/sw_school/DL2_data.log